In [1]:
import librosa
import neuro
import random
import numpy as np
import pandas as pd
import risp
import eons
import json
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import wave
import os 
from scipy.signal import find_peaks

# original params
if False:
    eo_params = {
        "starting_nodes": 3,
        "starting_edges": 6,
        "merge_rate": 0,
        "population_size": 100,
        "multi_edges": 0,
        "crossover_rate": 0.5,
        "mutation_rate": 0.9,
        "selection_type": "tournament",
        "tournament_size_factor": 0.1,
        "tournament_best_net_factor": 0.9,
        "random_factor": 0.05,
        "num_mutations": 3,
        "node_mutations": { "Threshold": 1.0 },
        "net_mutations": { },
        "edge_mutations": { "Weight": 0.5, "Delay": 0.5 },
        "num_best" : 4
    }

eo_params = {
    "starting_nodes": 3,
    "starting_edges": 6,
    "merge_rate": 0.1,
    "population_size": 100,
    "multi_edges": 0,
    "crossover_rate": 0.5,
    "mutation_rate": 0.8,
    "selection_type": "tournament",
    "tournament_size_factor": 0.1,
    "tournament_best_net_factor": 0.9,
    "random_factor": 0.05,
    "num_mutations": 4,
    "node_mutations": { "Threshold": 1.0 },
    "net_mutations": { },
    "edge_mutations": { "Weight": 0.5, "Delay": 0.5 },
    "num_best" : 4
}



In [2]:
import wave
import librosa
import numpy as np
import os 
from scipy.signal import find_peaks

unique_sample_rates = np.array([])
min_song_len = float('inf')

directory = 'audio_database'


def mel_binary_mean(mel_spectrum):
    # Computes the row-wise mean values (the mean of each channel)
    channel_means = np.mean(mel_spectrum, axis=1)

    # Applies thresholding to create binary representation
    binary_spectrum = np.where(mel_spectrum >= channel_means[:, np.newaxis], 1, 0)

    return binary_spectrum


def find_peaks_per_channel(spectrum, threshold=0):
    peaks = np.zeros_like(spectrum)
    
    for i in range(spectrum.shape[0]):
        channel_data = spectrum[i, :]
        channel_peaks, _ = find_peaks(channel_data, height=threshold)
        peaks[i, channel_peaks] = 1
    
    return peaks

# Create a new folder to save the npy files
new_folder = "npy_files"
new_folder_path = os.path.join(directory, new_folder)
os.makedirs(new_folder_path, exist_ok=True)

for folder_name in os.listdir(directory):
    unique_song_len = np.array([])

    folder_path = os.path.join(directory, folder_name)
    if os.path.isdir(folder_path):  # Check if the item is a directory
        for filename in os.listdir(folder_path):
            if filename.endswith('.wav'):
                try: 
                    file_path = os.path.join(folder_path, filename)
                    audio_signal, sample_rate = librosa.load(file_path, sr=None)

                    unique_sample_rates = np.append(unique_sample_rates, sample_rate)
                    
                    n_fft = 2048
                    hop_length = n_fft // 16
                    magnitude_spectrum = np.abs(librosa.stft(audio_signal, n_fft=n_fft, hop_length=hop_length))
                    num_mels = 8
                    mel_spectrum = librosa.feature.melspectrogram(
                        sr=sample_rate,
                        S=magnitude_spectrum,
                        n_fft=n_fft,
                        hop_length=hop_length,
                        n_mels=num_mels
                    )
                    #mel_spectrum = mel_spectrum[:, :min_song_len]
                    unique_song_len = np.append(unique_song_len, np.shape(mel_spectrum[0]))
                    peak_spectrogram = mel_binary_mean(mel_spectrum) #<---------- to change encoder 
                    #peak_spectrogram = peak_spectrogram[:, :min_song_len]
                    
                    if min_song_len > len(peak_spectrogram[1]):
                        min_song_len = len(peak_spectrogram[1])


                    output_filename = f"{filename}_peak_spectrogram.npy"
                    output_path = os.path.join(new_folder_path, output_filename)
                    np.save(output_path, peak_spectrogram)
                
                except Exception as e:
                    print(f"Error processing {filename} in folder {folder_name}: {str(e)}")
                    continue
        print(f"unique song length in {folder_name} is {np.unique(unique_song_len)}")

min_song_len = int(min_song_len)        
print(f"unique sample rates for all genres {np.unique(unique_sample_rates)}")
print(f"minimum song length is {min_song_len}")

unique song length in pop is [5169.]
unique song length in metal is [5169. 5171.]
unique song length in disco is [5167. 5169. 5170. 5171. 5184. 5189. 5196. 5205. 5219. 5220.]
unique song length in npy_files is []
unique song length in blues is [5171.]
unique song length in reggae is [5169. 5171.]
unique song length in classical is [5167. 5168. 5170. 5171. 5181. 5184. 5198. 5232. 5236. 5253.]
unique song length in rock is [5168. 5171. 5219. 5231. 5238.]
unique song length in hiphop is [5157. 5168. 5169. 5170. 5171. 5191. 5198. 5217. 5220. 5229. 5232. 5280.]
unique song length in country is [5165. 5168. 5171. 5183. 5186. 5210. 5226. 5232.]


/Users/quinn/Documents/local_framework/framework/pyframework/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


Error processing jazz.00054.wav in folder jazz: 
unique song length in jazz is [5170. 5171. 5172. 5198. 5203. 5210. 5215. 5229. 5251.]
unique sample rates for all genres [22050.]
minimum song length is 5157


In [3]:

def load_npy_files_with_prefix(directory, prefix, min_song_len):
    npy_files = [file for file in os.listdir(directory) if file.startswith(prefix) and file.endswith('.npy')]
    npy_files.sort()  # Sort the files for consistent order

    if len(npy_files) == 0:
        raise ValueError(f"No npy files found with prefix '{prefix}' in directory '{directory}'")
    loaded_data = []

    times_padded = 0
    for npy_file in npy_files:
        npy_path = os.path.join(directory, npy_file)
        data = np.load(npy_path)

        # Pad or trim the data array to the desired shape (min_song_len)
   
        if len(data[1]) > min_song_len:
            trimmed_data = data[:, :min_song_len]
            loaded_data.append(trimmed_data)
        else:
            loaded_data.append(data)

    return np.array(loaded_data)

In [4]:
directory = new_folder_path

# Loading hiphop songs
X_hiphop = load_npy_files_with_prefix(directory, 'hiphop', min_song_len=min_song_len)
y_hiphop = ['hiphop'] * len(X_hiphop)

# Loading country songs
X_country = load_npy_files_with_prefix(directory, 'country', min_song_len=min_song_len)
y_country = ['country'] * len(X_country)

# Combining the data and labels
X = np.concatenate((X_hiphop, X_country), axis=0)
y = np.concatenate((y_hiphop, y_country), axis=0)

X = (np.rint(X)).astype(int)





In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=7)

labels = np.unique(y_train)
dmin = [np.min(X_train[i]) for i in range(X_train.shape[0])]
dmax = [np.max(X_train[i]) for i in range(X_train.shape[0])]




# selecting sample scale 
for i in range(len(X_train)):
    # mel region (20 total) corresponds to id 
    for j in range(len(X_train[i])):
        # time bin selection 
        for k in range(len(X_train[i][j])): 
            if X_train[i][j][k] != 0:
                spike = neuro.Spike(id=j,time=0,value=X_train[i][j][k])
                proc .... 


In [ ]:
risp_config = {
  "leak_mode": True,
  "min_weight": -1,
  "max_weight": 1,
  "min_threshold": -1,
  "max_threshold": 1,
  "max_delay": 5
}

proc = risp.Processor(risp_config)

temp_net = neuro.Network()
temp_net.set_properties(proc.get_network_properties())

In [7]:
def create_neuron(neuron_id, net, moa):
    neuron = net.add_node(neuron_id)
    temp_net.randomize_node_properties(moa, neuron)
    return neuron

In [8]:
n_inputs = num_mels
n_hidden = 200
n_outputs = len(labels)
n_neurons = n_inputs+n_hidden+n_outputs
n_synapses = 1200
seed = 42

moa = neuro.MOA()
moa.seed(seed)
random.seed(seed)

In [9]:

for i in range(n_inputs):
    neuron = create_neuron(i, temp_net, moa)
    neuron.set("Threshold",0.75)
    temp_net.add_input(neuron.id)
    
for i in range(n_outputs):
    neuron = create_neuron(i+n_inputs, temp_net, moa)
    neuron.set("Threshold",0.75)
    temp_net.add_output(neuron.id)
    
for i in range(n_hidden):
    neuron = create_neuron(i+n_inputs+n_outputs, temp_net, moa)




In [10]:
for i in range(n_synapses):
    source = random.randint(0,n_neurons-1)
    dest = random.randint(0,n_neurons-1)
    synapse = temp_net.add_or_get_edge(source, dest)
    temp_net.randomize_edge_properties(moa, synapse)

In [11]:
evolver = eons.EONS(eo_params)
evolver.set_template_network(temp_net)

pop = evolver.generate_population(eo_params,1)

In [12]:
def get_prediction(x):
    #print(x)
    #proc.clear_activity()
    for j in range(len(x)):
            # time bin selection 
            for k in range(len(x[j])): 
                if x[j][k] != 0:
                    spike = neuro.Spike(id=j,time=0,value=x[j][k])
                    proc.apply_spike(spike)
    proc.run(50)
    return labels[proc.output_count_max(n_outputs)[0]]

In [13]:
def fitness(net, X, y):
    proc.load_network(net)
    
    # Set up output tracking
    for i in range(n_outputs):
        proc.track_neuron_events(i)
    
    y_predict = [get_prediction(x) for x in X]
    #print(len(y_predict))
    return accuracy_score(y_predict, y)

In [14]:
'''
# Define batch size and number of batches
batch_size = 10
n_batches = len(X_train) // batch_size

# Iterate over the batches
for epoch in range(100):
    # Track the best performing network throughout and print the current best result
    best_fitness = 0.0
    mean_fitness = 0.0

    # Process each batch
    for batch_idx in range(n_batches):
        # Get the batch data and labels
        batch_start = batch_idx * batch_size
        batch_end = (batch_idx + 1) * batch_size
        X_batch = X_train[batch_start:batch_end]
        y_batch = y_train[batch_start:batch_end]

        # Calculate the fitnesses of all the networks in the population for the batch
        fitnesses = [fitness(net.network, X_batch, y_batch) for net in pop.networks]

        # Update the best and mean fitness
        max_fit = max(fitnesses)
        mean_fit = np.mean(fitnesses)
        best_fitness = max(best_fitness, max_fit)
        mean_fitness += mean_fit
        print("Epoch:", epoch, "Batch:", batch_idx, "Best Fitness:", best_fitness, "Mean Fitness:", mean_fitness)

        # Create the next population based on the fitnesses of the current population for the batch
        pop = evolver.do_epoch(pop, fitnesses, eo_params)

    # Calculate the mean fitness across all batches
    mean_fitness /= n_batches

    # Print the progress for the epoch
    print("Epoch:", epoch, "Best Fitness:", best_fitness, "Mean Fitness:", mean_fitness)
'''


'\n# Define batch size and number of batches\nbatch_size = 10\nn_batches = len(X_train) // batch_size\n\n# Iterate over the batches\nfor epoch in range(100):\n    # Track the best performing network throughout and print the current best result\n    best_fitness = 0.0\n    mean_fitness = 0.0\n\n    # Process each batch\n    for batch_idx in range(n_batches):\n        # Get the batch data and labels\n        batch_start = batch_idx * batch_size\n        batch_end = (batch_idx + 1) * batch_size\n        X_batch = X_train[batch_start:batch_end]\n        y_batch = y_train[batch_start:batch_end]\n\n        # Calculate the fitnesses of all the networks in the population for the batch\n        fitnesses = [fitness(net.network, X_batch, y_batch) for net in pop.networks]\n\n        # Update the best and mean fitness\n        max_fit = max(fitnesses)\n        mean_fit = np.mean(fitnesses)\n        best_fitness = max(best_fitness, max_fit)\n        mean_fitness += mean_fit\n        print("Epoch:",

In [15]:
vals = []
for i in range(25):
    # Calculate the fitnesses of all of the networks in the population
    fitnesses = [fitness(net.network, X_train, y_train) for net in pop.networks]
    # Track the best performing network throughout and print the current best result
    max_fit = max(fitnesses)
    mean_fit = np.mean(fitnesses)
    #print(fitnesses)
    vals.append(max_fit)
    print("Epoch ", i, " : ","max fit", max_fit, "mean fit",mean_fit)
    
    # Create the next population based on the fitnesses of the current population
    pop = evolver.do_epoch(pop, fitnesses, eo_params)



Epoch  0  :  max fit 0.5149253731343284 mean fit 0.48567164179104483
Epoch  1  :  max fit 0.5149253731343284 mean fit 0.4910447761194031
Epoch  2  :  max fit 0.5149253731343284 mean fit 0.5080597014925374
Epoch  3  :  max fit 0.5149253731343284 mean fit 0.5098507462686568
Epoch  4  :  max fit 0.5149253731343284 mean fit 0.5098507462686568
Epoch  5  :  max fit 0.5149253731343284 mean fit 0.5101492537313433
Epoch  6  :  max fit 0.5149253731343284 mean fit 0.5109701492537314
Epoch  7  :  max fit 0.5149253731343284 mean fit 0.5113432835820896
Epoch  8  :  max fit 0.5149253731343284 mean fit 0.5109701492537314
Epoch  9  :  max fit 0.5149253731343284 mean fit 0.5119402985074628
Epoch  10  :  max fit 0.5149253731343284 mean fit 0.5116417910447761
Epoch  11  :  max fit 0.5149253731343284 mean fit 0.5125373134328358
Epoch  12  :  max fit 0.5149253731343284 mean fit 0.5119402985074627
Epoch  13  :  max fit 0.5149253731343284 mean fit 0.5116417910447761
Epoch  14  :  max fit 0.5149253731343284 me

In [ ]:

best_net = pop.networks[fitnesses.index(max_fit)].network
train = fitness(best_net, X_train, y_train)
print("Training Accuracy: ", train)
test = fitness(best_net, X_test, y_test)
print("Testing Accuracy: ", test)

Training Accuracy:  0.6194029850746269
Testing Accuracy:  0.5


After 1000 Epochs previous network acheived (to beat)
Training Accuracy:  0.6194029850746269
Testing Accuracy:  0.5